In [2]:
import cv2
import mediapipe as mp # mục đích sử dụng pose estimation
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 # đếm số lượt nâng tạ lên và xuống
stage = None # Down và Up

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image) #
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            print(angle)
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
                #         điểm BĐ, điểm KT,   màu       ,thickness
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                   #  Font chữ,          fontScale, màu chữ,thickness, ) https://www.geeksforgeeks.org/python-opencv-cv2-puttext-method/
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'): # nhấn q để tắt camera
            break

    cap.release()
    cv2.destroyAllWindows()


def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0]) # tính góc
    angle = np.abs(radians*180.0/np.pi) # đổi sang độ
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

146.64934689668863
152.51500408343068
155.16241760282043
155.3632394574851
156.03360212699934
158.20774220587015
158.4104890462305
158.61856038221336
159.32687962890296
160.0690598881268
162.9865691261486
164.9632361733538
164.1906982227741
162.93784967436642
162.93745778932544
167.34339337797837
164.38905506415398
170.47403385758096
96.82865382116188
156.19528537560748
152.84571547304404
152.53055393387262
151.04930816907816
150.39599713108555
149.7275293802086
148.96116958326428
148.43499788580368
145.98590895061216
145.09288301455322
152.1082212829969
154.98501478254235
152.55720277298735
152.2348833776756
154.2273072415825
156.59699116481366
160.83777498389836
163.13280057048763
164.4003164318686
166.00407276450838
166.1410108611326
166.46556346645565
166.21766505678818
166.40432689792493
166.42585367739557
166.6056886930136
166.31799964594538
166.28734830945447
166.40063724419807
166.48000336891062
166.4758239866465
166.75851902970084
166.6767251989863
166.26674236259106
165.88329